In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModel
import numpy as np

# Load Twitter data
twitter_data = pd.read_csv('../Data/twitter_data.csv')
twitter_data.dropna(subset=['tweet_content'], inplace=True)  # Drop rows where tweet_content is missing

# Download necessary NLTK resources
# nltk.download('stopwords')
# nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Text preprocessing function
def preprocess_text(text):
    # Tokenization and removing stopwords
    tokens = [word for word in text.lower().split() if word not in stop_words]
    # Stemming
    stemmed = [stemmer.stem(word) for word in tokens]
    # Lemmatization
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
    return ' '.join(lemmatized)

# Apply preprocessing
twitter_data['processed_content'] = twitter_data['tweet_content'].apply(preprocess_text)

# Count Vectorizer
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(twitter_data['processed_content'])

# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(twitter_data['processed_content'])

# Word2Vec embeddings
sentences = [tweet.split() for tweet in twitter_data['processed_content']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Function to get sentence vector by averaging Word2Vec embeddings
def get_sentence_vector(sentence, model):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

# Apply Word2Vec
twitter_data['word2vec_vector'] = twitter_data['processed_content'].apply(lambda x: get_sentence_vector(x, word2vec_model))

# Load GloVe embeddings
def load_glove_model(glove_file):
    glove_model = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            glove_model[word] = embedding
    return glove_model

# Specify the path to your downloaded GloVe embeddings
glove_file_path = 'C:/Users/HP/Downloads/glove.6B/glove.6B.100d.txt'  
glove_model = load_glove_model(glove_file_path)

# Function to get GloVe vector for a sentence
def get_glove_vector(sentence, model):
    words = sentence.split()
    word_vectors = [model[word] for word in words if word in model]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(100)  # 100d embeddings

# Apply GloVe embeddings
twitter_data['glove_vector'] = twitter_data['processed_content'].apply(lambda x: get_glove_vector(x, glove_model))

# Display processed data
print(twitter_data.head())



c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   tweetID       entity sentiment  \
0     2401  Borderlands  Positive   
1     2401  Borderlands  Positive   
2     2401  Borderlands  Positive   
3     2401  Borderlands  Positive   
4     2401  Borderlands  Positive   

                                       tweet_content  \
0  im getting on borderlands and i will murder yo...   
1  I am coming to the borders and I will kill you...   
2  im getting on borderlands and i will kill you ...   
3  im coming on borderlands and i will murder you...   
4  im getting on borderlands 2 and i will murder ...   

                 processed_content  \
0       im get borderland murder ,   
1            come border kill all,   
2      im get borderland kill all,   
3   im come borderland murder all,   
4  im get borderland 2 murder all,   

                                     word2vec_vector  \
0  [-1.0745261, 0.5151903, -0.5766882, -0.7825491...   
1  [-0.5513947, 0.3456574, -0.13432187, -0.499555...   
2  [-0.9943293, 0.40885267, -0.48143703, -0

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


# Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(twitter_data['processed_content'])
y = twitter_data['sentiment']  # Assuming 'sentiment' is the target variable

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Logistic Regression model
model = LogisticRegression(max_iter=1000)  # Increase max_iter if you encounter convergence warnings
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7827333333333333
Classification Report:
               precision    recall  f1-score   support

  Irrelevant       0.83      0.68      0.75      2666
    Negative       0.78      0.84      0.81      4464
     Neutral       0.81      0.74      0.77      3706
    Positive       0.74      0.83      0.78      4164

    accuracy                           0.78     15000
   macro avg       0.79      0.77      0.78     15000
weighted avg       0.79      0.78      0.78     15000

